In [7]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re
from datetime import datetime, timedelta
from collections import defaultdict
import os

def setup_driver():
    """브라우저 설정 및 시작"""
    print("🚀 브라우저 설정 중...")
    
    # Chrome 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    # User-Agent 설정
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    
    # 브라우저 크기 설정
    chrome_options.add_argument('--window-size=1920,1080')
    
    print("   ✅ Chrome 옵션 설정 완료")
    
    # 드라이버 생성
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        # 자동화 감지 방지
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        print("   ✅ Chrome 드라이버 시작 성공!")
        return driver
    except Exception as e:
        print(f"   ❌ 드라이버 시작 실패: {e}")
        return None

def login_to_naver(driver):
    """네이버 로그인"""
    print("\n🔐 네이버 로그인 시작...")
    
    try:
        # 네이버 로그인 페이지로 이동
        driver.get("https://nid.naver.com/nidlogin.login")
        time.sleep(3)
        
        print("   📍 네이버 로그인 페이지 접속 완료")
        print("   ⏳ 수동 로그인을 해주세요...")
        print("   📌 로그인 후 아무 키나 눌러주세요.")
        
        # 수동 로그인 대기
        input("   👆 로그인 완료 후 Enter를 눌러주세요...")
        
        # 로그인 확인
        current_url = driver.current_url
        if "naver.com" in current_url and "login" not in current_url:
            print("   ✅ 로그인 성공!")
            return True
        else:
            print("   ⚠️  로그인 상태 확인 중...")
            time.sleep(2)
            return True  # 일단 진행
            
    except Exception as e:
        print(f"   ❌ 로그인 과정 오류: {e}")
        return False

def extract_post_content(driver, post_data):
    """개별 게시글에서 텍스트와 이미지 추출"""
    url = post_data['url']
    try:
        print(f"      본문 추출: {post_data['title'][:30]}...")
        driver.get(url)
        time.sleep(3)
        
        # 카페 프레임으로 전환
        try:
            driver.switch_to.frame("cafe_main")
            time.sleep(2)
        except:
            pass
        
        # 페이지 로딩 대기
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "article_viewer"))
            )
        except:
            pass
        
        # JavaScript로 텍스트와 이미지 동시 추출
        extract_js = """
        var result = {
            text_content: '',
            image_urls: [],
            image_data: []
        };
        
        try {
            // 본문 텍스트 추출
            var textSelectors = [
                '.article_viewer',
                '.se-main-container',
                '.content-area',
                '.article-content',
                '.CafeViewer'
            ];
            
            var textFound = false;
            for (var i = 0; i < textSelectors.length && !textFound; i++) {
                var textElement = document.querySelector(textSelectors[i]);
                if (textElement) {
                    // 텍스트만 추출 (이미지 제외)
                    var textNodes = [];
                    var walker = document.createTreeWalker(
                        textElement,
                        NodeFilter.SHOW_TEXT,
                        null,
                        false
                    );
                    
                    var node;
                    while (node = walker.nextNode()) {
                        var text = node.textContent.trim();
                        if (text && 
                            !text.includes('더보기') && 
                            !text.includes('첨부파일') &&
                            text.length > 1) {
                            textNodes.push(text);
                        }
                    }
                    
                    result.text_content = textNodes.join(' ').trim();
                    textFound = true;
                }
            }
            
            // 이미지 URL 추출
            var imageSelectors = [
                '.se-image-resource',
                '.se-module-image img',
                '.article_viewer img',
                '.se-main-container img',
                'img[src*="cafeptthumb"]',
                'img[src*="blogfiles"]',
                'img[src*="pstatic"]'
            ];
            
            var foundImages = new Set();
            
            imageSelectors.forEach(function(selector) {
                var images = document.querySelectorAll(selector);
                images.forEach(function(img) {
                    var src = img.src || img.getAttribute('data-src') || img.getAttribute('data-original');
                    
                    if (src && 
                        (src.includes('cafeptthumb') || 
                         src.includes('blogfiles') || 
                         src.includes('pstatic') ||
                         src.includes('jpeg') || 
                         src.includes('jpg') || 
                         src.includes('png') || 
                         src.includes('gif') ||
                         src.includes('webp')) &&
                        !src.includes('emoticon') &&
                        !src.includes('icon') &&
                        !foundImages.has(src)) {
                        
                        foundImages.add(src);
                        
                        var imageInfo = {
                            url: src,
                            alt: img.alt || '',
                            width: img.naturalWidth || img.width || 0,
                            height: img.naturalHeight || img.height || 0
                        };
                        
                        // 원본 URL 찾기
                        var linkElement = img.closest('a');
                        if (linkElement && linkElement.getAttribute('data-linkdata')) {
                            try {
                                var linkData = JSON.parse(linkElement.getAttribute('data-linkdata'));
                                if (linkData.src) {
                                    imageInfo.original_url = linkData.src;
                                    imageInfo.original_width = linkData.originalWidth;
                                    imageInfo.original_height = linkData.originalHeight;
                                }
                            } catch(e) {}
                        }
                        
                        result.image_urls.push(src);
                        result.image_data.push(imageInfo);
                    }
                });
            });
            
            console.log('텍스트 길이:', result.text_content.length);
            console.log('이미지 개수:', result.image_urls.length);
            
        } catch(e) {
            console.error('추출 오류:', e);
        }
        
        return result;
        """
        
        # JavaScript 실행
        extraction_result = driver.execute_script(extract_js)
        
        # 결과 처리
        text_content = extraction_result.get('text_content', '').strip()
        image_urls = extraction_result.get('image_urls', [])
        image_data = extraction_result.get('image_data', [])
        
        # 기본 프레임으로 복귀
        try:
            driver.switch_to.default_content()
        except:
            pass
        
        # 이미지 정보를 문자열로 변환
        image_urls_str = '|'.join(image_urls) if image_urls else ''
        image_details_str = ''
        
        if image_data:
            image_details = []
            for img_info in image_data:
                detail = f"URL:{img_info.get('url', '')}"
                if img_info.get('original_url'):
                    detail += f";ORIGINAL:{img_info.get('original_url', '')}"
                if img_info.get('width'):
                    detail += f";SIZE:{img_info.get('width', 0)}x{img_info.get('height', 0)}"
                if img_info.get('alt'):
                    detail += f";ALT:{img_info.get('alt', '')}"
                image_details.append(detail)
            image_details_str = '||'.join(image_details)
        
        # 데이터 업데이트 (기존 텍스트가 없을 때만 업데이트)
        updates = {}
        
        if not post_data.get('text_content') or pd.isna(post_data.get('text_content')):
            updates['text_content'] = text_content
            updates['content_length'] = len(text_content)
        
        # 이미지 정보는 항상 업데이트
        updates.update({
            'image_urls': image_urls_str,
            'image_details': image_details_str,
            'image_count': len(image_urls),
            'has_images': 'Yes' if image_urls else 'No'
        })
        
        post_data.update(updates)
        
        print(f"      ✅ 텍스트: {len(text_content)}자, 이미지: {len(image_urls)}개")
        
        return post_data
        
    except Exception as e:
        print(f"      ❌ 본문 추출 실패: {str(e)[:50]}...")
        # 실패한 경우에도 이미지 관련 필드는 업데이트
        post_data.update({
            'image_urls': post_data.get('image_urls', ''),
            'image_details': post_data.get('image_details', ''),
            'image_count': post_data.get('image_count', 0),
            'has_images': post_data.get('has_images', 'No')
        })
        return post_data

def download_images(driver, image_urls_str, post_id, download_folder="images"):
    """이미지 다운로드 함수"""
    if not image_urls_str:
        return []
    
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)
    
    image_urls = image_urls_str.split('|')
    downloaded_files = []
    
    for i, img_url in enumerate(image_urls):
        try:
            print(f"        이미지 다운로드: {i+1}/{len(image_urls)}")
            
            # 간단한 방법: requests 사용
            import requests
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            }
            
            response = requests.get(img_url, headers=headers, timeout=10)
            if response.status_code == 200:
                # 파일명 생성
                file_ext = img_url.split('.')[-1].split('?')[0]
                if file_ext not in ['jpg', 'jpeg', 'png', 'gif', 'webp']:
                    file_ext = 'jpg'
                
                filename = f"{post_id}_img_{i+1}.{file_ext}"
                filepath = os.path.join(download_folder, filename)
                
                # 파일 저장
                with open(filepath, 'wb') as f:
                    f.write(response.content)
                
                downloaded_files.append(filepath)
                print(f"        ✅ 저장: {filename}")
            
        except Exception as e:
            print(f"        ❌ 이미지 다운로드 실패: {str(e)[:30]}...")
            continue
        
        time.sleep(0.5)  # 다운로드 간 딜레이
    
    return downloaded_files

def find_posts_without_images(csv_file):
    """이미지가 없는 게시글 찾기"""
    print(f"\n📋 CSV 파일 분석 중: {csv_file}")
    
    try:
        df = pd.read_csv(csv_file, encoding='utf-8-sig')
        print(f"   ✅ 총 {len(df)}개 게시글 로드됨")
        
        # 이미지가 없는 게시글 찾기
        # 여러 조건으로 확인
        conditions = []
        
        # 조건 1: image_urls가 비어있거나 NaN
        if 'image_urls' in df.columns:
            conditions.append(df['image_urls'].isna() | (df['image_urls'] == '') | (df['image_urls'] == '0'))
        
        # 조건 2: image_count가 0이거나 NaN
        if 'image_count' in df.columns:
            conditions.append(df['image_count'].isna() | (df['image_count'] == 0))
        
        # 조건 3: has_images가 'No'이거나 NaN
        if 'has_images' in df.columns:
            conditions.append(df['has_images'].isna() | (df['has_images'] == 'No') | (df['has_images'] == ''))
        
        # 모든 조건을 OR로 결합
        if conditions:
            mask = conditions[0]
            for condition in conditions[1:]:
                mask = mask | condition
            
            posts_without_images = df[mask].copy()
        else:
            # 이미지 관련 컬럼이 없으면 모든 게시글 대상
            posts_without_images = df.copy()
        
        print(f"   📊 이미지 추출이 필요한 게시글: {len(posts_without_images)}개")
        print(f"   📊 이미지가 있는 게시글: {len(df) - len(posts_without_images)}개")
        
        if len(posts_without_images) > 0:
            print(f"\n💡 재크롤링 대상 샘플:")
            for i, row in posts_without_images.head(3).iterrows():
                print(f"   {i+1}. {row.get('title', 'No Title')[:40]}...")
                print(f"      URL: {row.get('url', 'No URL')}")
                print(f"      현재 이미지: {row.get('image_count', 'N/A')}개")
        
        return posts_without_images.to_dict('records')
        
    except Exception as e:
        print(f"   ❌ CSV 파일 읽기 실패: {e}")
        return []

def recrawl_images(driver, posts_data, download_images_choice=False):
    """이미지가 없는 게시글들의 이미지 재크롤링"""
    if not posts_data:
        print("❌ 재크롤링할 게시글이 없습니다")
        return []
    
    print(f"\n🔄🔄🔄 이미지 재크롤링 시작! 🔄🔄🔄")
    print(f"📊 대상 게시글: {len(posts_data)}개")
    
    updated_posts = []
    success_count = 0
    fail_count = 0
    
    for i, post in enumerate(posts_data, 1):
        print(f"\n[{i}/{len(posts_data)}] 이미지 재추출 중...")
        print(f"   제목: {post.get('title', 'No Title')[:50]}...")
        
        try:
            # 이미지 추출
            updated_post = extract_post_content(driver, post.copy())
            
            # 이미지가 발견되었는지 확인
            new_image_count = updated_post.get('image_count', 0)
            
            if new_image_count > 0:
                print(f"   🎉 새로운 이미지 {new_image_count}개 발견!")
                success_count += 1
                
                # 이미지 다운로드
                if download_images_choice and updated_post.get('image_urls'):
                    year = updated_post.get('year', 'unknown')
                    download_folder = f"images_recrawl/{year}"
                    post_id = f"recrawl_{year}_{i:04d}"
                    
                    print(f"      📥 이미지 다운로드 중...")
                    downloaded_files = download_images(
                        driver, 
                        updated_post['image_urls'], 
                        post_id, 
                        download_folder
                    )
                    updated_post['downloaded_images'] = '|'.join(downloaded_files)
                
            else:
                print(f"   ⚠️  이미지 없음 (원래대로)")
                fail_count += 1
            
            updated_posts.append(updated_post)
            
            # 중간 저장 (50개마다)
            if i % 50 == 0:
                temp_df = pd.DataFrame(updated_posts)
                temp_filename = f"temp_recrawl_{i}.csv"
                temp_df.to_csv(temp_filename, index=False, encoding='utf-8-sig')
                print(f"   💾 중간 저장: {temp_filename}")
            
            # 서버 부하 방지
            time.sleep(1)
            
        except Exception as e:
            print(f"   ❌ 재추출 실패: {str(e)[:50]}...")
            # 실패해도 원본 데이터는 유지
            updated_posts.append(post)
            fail_count += 1
            continue
    
    print(f"\n🏁 이미지 재크롤링 완료!")
    print(f"   ✅ 성공: {success_count}개")
    print(f"   ❌ 실패/이미지없음: {fail_count}개")
    
    return updated_posts

def merge_and_save_results(original_csv, updated_posts, cafe_id):
    """원본 CSV와 업데이트된 데이터를 병합하여 저장"""
    print(f"\n🔄 데이터 병합 중...")
    
    try:
        # 원본 CSV 로드
        original_df = pd.read_csv(original_csv, encoding='utf-8-sig')
        print(f"   📋 원본 데이터: {len(original_df)}개")
        
        # 업데이트된 데이터를 DataFrame으로 변환
        updated_df = pd.DataFrame(updated_posts)
        print(f"   🔄 업데이트된 데이터: {len(updated_df)}개")
        
        # URL을 기준으로 병합
        if 'url' in original_df.columns and 'url' in updated_df.columns:
            # 업데이트된 데이터의 URL들
            updated_urls = set(updated_df['url'].tolist())
            
            # 원본에서 업데이트되지 않은 데이터
            unchanged_df = original_df[~original_df['url'].isin(updated_urls)].copy()
            
            # 병합
            final_df = pd.concat([unchanged_df, updated_df], ignore_index=True)
            
            print(f"   ✅ 병합 완료:")
            print(f"      - 기존 데이터: {len(unchanged_df)}개")
            print(f"      - 업데이트된 데이터: {len(updated_df)}개")
            print(f"      - 최종 데이터: {len(final_df)}개")
        else:
            print("   ⚠️  URL 컬럼이 없어 단순 결합")
            final_df = pd.concat([original_df, updated_df], ignore_index=True)
        
        # 중복 제거
        if 'url' in final_df.columns:
            original_count = len(final_df)
            final_df = final_df.drop_duplicates(subset=['url'], keep='last')  # 최신 데이터 유지
            removed_count = original_count - len(final_df)
            
            if removed_count > 0:
                print(f"   🔄 중복 제거: {removed_count}개")
        
        # 컬럼 순서 정리
        column_order = [
            'year', 'parsed_date', 'date', 'title', 'author', 'views', 
            'text_content', 'content_length', 'has_images', 'image_count', 
            'image_urls', 'image_details', 'downloaded_images', 'url', 'page'
        ]
        
        # 존재하는 컬럼만 선택
        available_columns = [col for col in column_order if col in final_df.columns]
        final_df = final_df[available_columns]
        
        # 날짜순 정렬
        if 'parsed_date' in final_df.columns:
            df_with_date = final_df[~final_df['parsed_date'].str.contains('매칭|Unknown', na=False)].copy()
            df_no_date = final_df[final_df['parsed_date'].str.contains('매칭|Unknown', na=False)].copy()
            
            if len(df_with_date) > 0:
                df_with_date = df_with_date.sort_values('parsed_date', ascending=False)
                final_df = pd.concat([df_with_date, df_no_date], ignore_index=True)
        
        # 최종 파일 저장
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        final_filename = f"cafe_{cafe_id}_posts_2023_2024_UPDATED_{timestamp}.csv"
        final_df.to_csv(final_filename, index=False, encoding='utf-8-sig')
        
        print(f"\n🏆 업데이트 완료!")
        print(f"📁 최종 파일: {final_filename}")
        print(f"📊 최종 데이터: {len(final_df)}개")
        
        # 이미지 통계
        if 'image_count' in final_df.columns:
            total_images = final_df['image_count'].sum()
            posts_with_images = final_df[final_df['image_count'] > 0].shape[0]
            posts_without_images = len(final_df) - posts_with_images
            
            print(f"\n📈 최종 이미지 통계:")
            print(f"   📸 총 이미지 수: {total_images}개")
            print(f"   📸 이미지 포함 게시글: {posts_with_images}개")
            print(f"   📝 텍스트만 게시글: {posts_without_images}개")
            
            if posts_with_images > 0:
                avg_images = total_images / posts_with_images
                print(f"   📸 평균 이미지 수: {avg_images:.1f}개/게시글")
        
        return final_filename
        
    except Exception as e:
        print(f"   ❌ 병합 실패: {e}")
        return None

def main():
    """이미지 재크롤링 메인 함수"""
    print("🔄🔄🔄 네이버 카페 이미지 재크롤링 도구 🔄🔄🔄")
    print("✨ 기존 CSV에서 이미지가 없는 게시글만 다시 크롤링합니다")
    
    # CSV 파일 경로 입력
    csv_file = input("\n📁 기존 CSV 파일 경로를 입력하세요: ").strip()
    
    if not os.path.exists(csv_file):
        print(f"❌ 파일을 찾을 수 없습니다: {csv_file}")
        return
    
    # 카페 ID 추출 (파일명에서)
    cafe_id = "12730407"  # 기본값
    if "cafe_" in csv_file:
        try:
            cafe_id = csv_file.split("cafe_")[1].split("_")[0]
            print(f"   ✅ 파일명에서 카페 ID 추출: {cafe_id}")
        except:
            print(f"   ⚠️  카페 ID 추출 실패, 기본값 사용: {cafe_id}")
    
    # 이미지가 없는 게시글 찾기
    posts_without_images = find_posts_without_images(csv_file)
    
    if not posts_without_images:
        print("✅ 모든 게시글에 이미지 정보가 있습니다!")
        return
    
    # 이미지 다운로드 여부
    download_images_choice = input(f"\n📥 이미지 파일도 다운로드하시겠습니까? (y/n): ").lower() == 'y'
    
    driver = None
    
    try:
        # 1단계: 브라우저 설정
        driver = setup_driver()
        if not driver:
            print("❌ 브라우저 시작 실패")
            return
        
        # 2단계: 네이버 로그인
        if not login_to_naver(driver):
            print("❌ 로그인 실패")
            return
        
        # 3단계: 이미지 재크롤링
        updated_posts = recrawl_images(driver, posts_without_images, download_images_choice)
        
        if not updated_posts:
            print("❌ 재크롤링 실패")
            return
        
        # 4단계: 결과 병합 및 저장
        final_file = merge_and_save_results(csv_file, updated_posts, cafe_id)
        
        if final_file:
            print(f"\n✅✅✅ 이미지 재크롤링 완료! ✅✅✅")
            print(f"✅ 기존 데이터와 새로운 이미지 데이터가 병합되었습니다")
            print(f"✅ 최종 파일: {final_file}")
        else:
            print("❌ 최종 저장 실패")
        
    except KeyboardInterrupt:
        print("\n⚠️  사용자가 작업을 중단했습니다")
    except Exception as e:
        print(f"\n💥 예상치 못한 오류: {e}")
    finally:
        # 브라우저 종료
        if driver:
            print("\n🔚 브라우저 종료 중...")
            try:
                driver.quit()
                print("   ✅ 브라우저 종료 완료")
            except:
                print("   ⚠️  브라우저 종료 중 오류")

if __name__ == "__main__":
    main()

🔄🔄🔄 네이버 카페 이미지 재크롤링 도구 🔄🔄🔄
✨ 기존 CSV에서 이미지가 없는 게시글만 다시 크롤링합니다

📋 CSV 파일 분석 중: /Users/sia/Desktop/main/Digital_Data/FinalProject/Codes/day3/temp_posts_2023_2024.csv
   ✅ 총 1162개 게시글 로드됨
   📊 이미지 추출이 필요한 게시글: 1162개
   📊 이미지가 있는 게시글: 0개

💡 재크롤링 대상 샘플:
   1. 브라이튼 여의도 아파트 잔여세대 분양 (광고)...
      URL: https://cafe.naver.com/f-e/cafes/12730407/articles/5808734?boardtype=L&menuid=199&referrerAllArticles=false&page=1
      현재 이미지: N/A개
   2. [LG전자 베스트샵 강남본점]팝업스토어 GRAND OPEN! (광고)...
      URL: https://cafe.naver.com/f-e/cafes/12730407/articles/5806398?boardtype=L&menuid=199&referrerAllArticles=false&page=1
      현재 이미지: N/A개
   3. 실제로 대출받기 : 전세자금대출 2편 [문준]...
      URL: https://cafe.naver.com/f-e/cafes/12730407/articles/5810898?boardtype=L&menuid=199&referrerAllArticles=false&page=1
      현재 이미지: N/A개
🚀 브라우저 설정 중...
   ✅ Chrome 옵션 설정 완료
   ✅ Chrome 드라이버 시작 성공!

🔐 네이버 로그인 시작...
   📍 네이버 로그인 페이지 접속 완료
   ⏳ 수동 로그인을 해주세요...
   📌 로그인 후 아무 키나 눌러주세요.
   ✅ 로그인 성공!

🔄🔄🔄 이미지 재크롤링 시작! 🔄🔄🔄
📊 대상 게시글: 1162개

[1